In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
modelo=pd.read_csv('data/modelo.csv',low_memory=False)

In [3]:
modelo.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
modelo.shape

(2341681, 23)

In [5]:
modelo.isnull().sum()

timestamp                         0
event                             0
person                            0
url                         2150550
sku                         1021151
model                       1020168
condition                   1021151
storage                     1021151
color                       1021151
skus                        1835732
search_term                 2227918
staticpage                  2330480
campaign_source             2150395
search_engine               2235275
channel                     2137612
new_vs_returning            2137612
city                        2137612
region                      2137612
country                     2137612
device_type                 2137612
screen_resolution           2137615
operating_system_version    2137612
browser_version             2137612
dtype: int64

In [6]:
modelo.drop(['url', 'sku', 'skus','channel','staticpage','condition','search_engine','device_type','campaign_source','country','region','search_term','new_vs_returning','city','region','screen_resolution','operating_system_version','browser_version'], axis=1, inplace=True)

In [7]:
training= pd.read_csv('data/training.csv',low_memory=False)

# Armado del modelo

In [8]:
import calendar
modelo['timestamp'] = pd.to_datetime(modelo['timestamp'])

### Fechas

In [9]:
modelo['dia']=modelo['timestamp'].dt.dayofyear
modelo['dia_de_la_semana']=modelo['timestamp'].dt.dayofweek
modelo['nombre_del_dia']=modelo['timestamp'].dt.day_name()
modelo['hora']=modelo['timestamp'].dt.hour

### Horas distintas

In [10]:
modelo['horas_distintas']=modelo['person'].map(modelo.groupby('person')['hora'].nunique())

### Eventos en los ultimos dias

### Tipos de eventos por persona

In [11]:
df_aux=modelo.groupby(['person','event'])['event'].size().unstack('event')
modelo=pd.merge(modelo,df_aux,on='person',how='left')

### Dias distintos en el ultimo mes

In [12]:
modelo['dias_distintos'] = modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) ] )['dia'].nunique().unstack()[True]) 

In [13]:
modelo['dias_distintos_ultima_semana']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5)&(modelo['timestamp'].dt.day>23) ] )['dia'].nunique().unstack()[True])  

### Dias distintos de la semana

In [14]:
modelo['dias_distintos_de_la_semana']=modelo['person'].map(modelo.groupby('person')['dia_de_la_semana'].nunique())    

In [15]:
modelo['dias_distintos_de_la_semana_last_month']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) ])['dia_de_la_semana'].nunique().unstack()[True])

### Ultima visita

In [16]:
modelo['ultima_visita']=modelo['person'].map(modelo.groupby('person' )['dia'].max())  

### Compras por dia de la semana

In [17]:
df_aux=modelo.groupby(['person','nombre_del_dia']).size().reset_index()
compras_por_dia_semana=modelo.loc[modelo['event']=='conversion']['nombre_del_dia'].value_counts().to_dict()
df_aux['compras_por_dia_semana']=df_aux[0]*df_aux['nombre_del_dia'].apply(lambda x: compras_por_dia_semana.get(x))
modelo['compras_por_dia_semana']=modelo['person'].map(df_aux.groupby('person')['compras_por_dia_semana'].sum())  

### Modelos y marcas de celuares distintos vistos

In [18]:
modelo['modelos_distintos_vistos']= modelo['person'].map( modelo.groupby('person')['model'].nunique())

modelo['marca']=modelo['model'].str.split().str.get(0)
modelo['marcas_distintas']=modelo['person'].map(modelo.groupby('person')['marca'].nunique()) 

### Visitas maximas de un mismo celular

In [19]:
modelo['celular_mas_visto']=modelo['person'].map(modelo.groupby(['person','model']).size().groupby('person').max()) 

### Relacion de colores por modelo

In [20]:
modelo['color']=modelo['color'].fillna(0)
modelo['modelos_por_color']=modelo['person'].map(modelo.groupby('person')['model'].nunique()/(modelo.groupby('person')['color'].nunique()))

### Eventos distintos

In [21]:
modelo['eventos_distintos']=modelo['person'].map(modelo.groupby('person')['event'].nunique())   

### Relacion de eventos sobre el total del usuario

In [22]:
#df_aux = modelo.groupby(['person','event']).size().unstack().fillna(0).astype(int)
#df_aux = df_aux.div(df_aux.sum(axis=1), axis=0)
#modelo = modelo.merge(df_aux, how='left', on='person')
#modelo.head()
#df_aux.head()

### Campaña en la ultima semana

In [23]:
modelo['campaña_ultima_semana']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) & (modelo['timestamp'].dt.day>23) & (modelo['event']=="ad campaign hit")]).size().unstack()[True]) 

### Visitas

In [24]:
modelo['total_visitas_usuario'] = modelo.groupby('person')['person'].transform('count')

modelo['visitas_ultimo_mes']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month)==5]).size().unstack()[True])

modelo['visitas_ultimos_15']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>15)]).size().unstack()[True])

modelo['visitas_ultima_semana']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True])

modelo['visitas_ultimos_3']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>27)]).size().unstack()[True])

### Checkouts

In [25]:
modelo['total_checkout']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['checkout'])

modelo['checkouts_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['checkout'])   

modelo['checkouts_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['checkout'])

### Compras

In [26]:
modelo['total_compras']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['conversion'])

modelo['compras_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['conversion'])

modelo['compras_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['conversion'])

### Subscripciones

In [27]:
modelo['subscripciones']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['lead'])

### Celulares mas vendidos

In [28]:
"""compras_por_modelo = modelo.groupby('event').get_group('conversion')['model'].value_counts().to_frame()
aux_df = modelo.groupby(['person', 'model']).size().reset_index().merge(compras_por_modelo, how='left', left_on='model', right_index=True)
aux_df['vistas_sobre_compras'] = aux_df[0] / aux_df['model_y']
aux_df = aux_df.groupby('person')['vistas_sobre_compras'].mean().reset_index()
modelo = modelo.merge(aux_df, how='left', on='person')
modelo.head()"""

"compras_por_modelo = modelo.groupby('event').get_group('conversion')['model'].value_counts().to_frame()\naux_df = modelo.groupby(['person', 'model']).size().reset_index().merge(compras_por_modelo, how='left', left_on='model', right_index=True)\naux_df['vistas_sobre_compras'] = aux_df[0] / aux_df['model_y']\naux_df = aux_df.groupby('person')['vistas_sobre_compras'].mean().reset_index()\nmodelo = modelo.merge(aux_df, how='left', on='person')\nmodelo.head()"

### Eventos por dia de la semana

In [29]:
"""dias = modelo.groupby('person')['nombre_del_dia'].value_counts().unstack().fillna(0).astype(int)
dias = dias.div(dias.sum(axis=1), axis=0)

modelo = modelo.merge(dias, how='left', on='person')
modelo.head()"""

"dias = modelo.groupby('person')['nombre_del_dia'].value_counts().unstack().fillna(0).astype(int)\ndias = dias.div(dias.sum(axis=1), axis=0)\n\nmodelo = modelo.merge(dias, how='left', on='person')\nmodelo.head()"

In [30]:
precios=pd.read_csv('precios.csv',low_memory=False)
precios['Storage'] = precios['Capacidad'].astype(str) + 'GB' 
precios['mod+stg']= precios['Model']+precios['Storage']
modelo['mod+stg']= modelo['model']+modelo['storage']
modelo=pd.merge(modelo,precios,on='mod+stg',how="outer")
modelo.drop(['mod+stg','Model','Storage','Capacidad'], axis=1,inplace=True)
modelo['Precio']=modelo['Precio'].fillna(modelo['Precio'].mean())

FileNotFoundError: File b'precios.csv' does not exist

In [ ]:
modelo['precio_max']=modelo['person'].map(modelo.groupby('person')['Precio'].max())
modelo['precio_mean']=modelo['person'].map(modelo.groupby('person')['Precio'].mean())
modelo['precio_std']=modelo['person'].map(modelo.groupby('person')['Precio'].std())
modelo['precio_min']=modelo['person'].map(modelo.groupby('person')['Precio'].min())

### Dropeo de columnas

In [ ]:
modelo.drop(['color','dia','dia_de_la_semana','nombre_del_dia','model','marca','timestamp','event','hora','Precio','storage'],1,inplace=True)
modelo=modelo.drop_duplicates(subset=['person'])

In [ ]:
modelo=modelo.fillna(0)

In [ ]:
modelo=pd.merge(modelo, training, on='person', how='left')
modelo.iloc[:,1:-1]=modelo.iloc[:,1:-1].astype(np.int64)
modelo.head()

In [ ]:
modelo.info()

## Test de datos a predecir en Kaggle

In [ ]:
a_predecir=modelo[modelo['label'].isnull()]
a_predecir.head()

## Modelo a entrenar

In [ ]:
modelo.drop(modelo[modelo['label'].isnull()].index,inplace=True)
modelo['label']=modelo['label'].astype(int)
modelo.head()

## Analisis

In [ ]:
len(modelo[modelo['label']==1]),len(modelo[modelo['label']==0]) 

In [ ]:
len(modelo[modelo['label']==1])*100/len(modelo['label']),len(modelo[modelo['label']==0])*100/len(modelo['label'])

In [ ]:
modelo[modelo['label']==1].head()

In [ ]:
modelo[modelo['label']==0].head()

In [ ]:
modelo[modelo['label']==1].mean()-modelo[modelo['label']==0].mean()

In [ ]:
modelo[modelo['label']==1]['ultima_visita'].plot(kind='hist',figsize=(24,12),bins=150)
plt.title('Ultima visita de usuarios que compraron en los proximos 15 dias')

In [ ]:
modelo[modelo['label']==0]['ultima_visita'].plot(kind='hist',figsize=(24,12),bins=150)
plt.title('Ultima visita de usuarios que no compraron en los proximos 15 dias')

### Usuarios cuya ultima visita es antes de mayo solo habian entrado una vez

In [ ]:
modelo.loc[(modelo['ultima_visita']<120)&(modelo['label']==1)]['dias_distintos'].nunique() 

In [ ]:
modelo.loc[(modelo['ultima_visita']<120)&(modelo['label']==0)]['dias_distintos'].nunique() 

### Cantidad de visitas por usuario

In [ ]:
modelo[modelo['label']==1]['total_visitas_usuario'].plot(kind='hist',figsize=(24,12),bins=150)
plt.title('Visitas de usuarios que van a comprar')
plt.yscale('log', nonposy='clip')

In [ ]:
modelo[modelo['label']==0]['total_visitas_usuario'].plot(kind='hist',figsize=(24,12),bins=150)
plt.title('Visitas de usuarios no que van a comprar')
plt.yscale('log', nonposy='clip')

### Cantidad de visitas no decide si va a comprar o no

In [ ]:
visitas={}
for i in range(100,3500,200):
    visitas[i]=[len(modelo.loc[(modelo['label']==0)&(modelo['total_visitas_usuario']>i)]),len(modelo.loc[(modelo['label']==1)&(modelo['total_visitas_usuario']>i)])]
visitas=pd.DataFrame(visitas)
visitas

### Cantidad de usuarios que visitaron por ultima vez la pagina antes de mayo, y realizaron una compra en los proximos 15 dias

In [ ]:
len(modelo.loc[(modelo['ultima_visita']<120)&(modelo['label']==1)]) 

In [ ]:
len(modelo.loc[(modelo['ultima_visita']<120)&(modelo['label']==0)]) 

### Cantidad de usuarios que visitaron por ultima vez la pagina en de mayo, y realizaron una compra en los proximos 15 dias

In [ ]:
len(modelo.loc[(modelo['ultima_visita']>119)&(modelo['label']==1)]) 

In [ ]:
len(modelo.loc[(modelo['ultima_visita']>119)&(modelo['label']==0)]) 

### Usuarios que van a comprar tienen mayor promedio de modelos vistos

In [ ]:
modelo[modelo['label']==1]['modelos_distintos_vistos'].mean(),modelo[modelo['label']==0]['modelos_distintos_vistos'].mean()

### Promedio de usuarios que van a comprar es mayor si ingresaron a traves de una campaña 

In [ ]:
modelo[modelo['label']==1]['campaña_ultima_semana'].mean(),modelo[modelo['label']==0]['campaña_ultima_semana'].mean()

### Un 0.44% de las personas que van a comprar entraron a traves de una campaña

In [ ]:
len(modelo.loc[(modelo['label']==1)&(modelo['campaña_ultima_semana']!=0)]),len(modelo.loc[(modelo['label']==0)&(modelo['campaña_ultima_semana']!=0)] )

In [ ]:
100*len(modelo.loc[(modelo['label']==1)&(modelo['campaña_ultima_semana']!=0)]) /len(modelo[modelo['label']==1]),100*len(modelo.loc[(modelo['label']==0)&(modelo['campaña_ultima_semana']!=0)]) /len(modelo[modelo['label']==0])

De las 432, 90 (20%) ya habian comprado. O mejor dicho, un 80% que no habia comprado, a traves de una campaña despues terminó comprando.

In [ ]:
len(modelo.loc[(modelo['label']==1)&(modelo['campaña_ultima_semana']!=0)&(modelo['total_compras']!=0)])

### Un 20% de usuarios que van a comprar, ya lo habian hecho

In [ ]:
len(modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)]), 100*len(modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)])/len(modelo[modelo['label']==1])

In [ ]:
modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)]['total_compras'].plot(kind='hist',figsize=(24,12),bins=30)
plt.title('Frecuencia de compras anteriores de usuarios que van a comprar')

28 (2.8%) usuarios ya habian comprado antes de mayo. 

In [ ]:
len(modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)&(modelo['ultima_visita']<120)])

Pero estos usuarios tenian una media de 129 visitas a la pagina

In [ ]:
modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)&(modelo['ultima_visita']<120)]['total_visitas_usuario'].mean()

De los otros 169, su media de visitas a la pagina es de 240! Esto demuestra una tendencia a comprar.

In [ ]:
modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)&(modelo['ultima_visita']>120)]['total_visitas_usuario'].mean()

Y su media de dias distintos es mayor a 6.

In [ ]:
modelo.loc[(modelo['label']==1)&(modelo['total_compras']>0)&(modelo['ultima_visita']>120)]['dias_distintos'].mean()

141 es la media de aquellos que ya habian comprado, pero no lo hicieron luego.

In [ ]:
modelo.loc[(modelo['label']==0)&(modelo['total_compras']>0)&(modelo['ultima_visita']>120)]['total_visitas_usuario'].mean()

Y para estos, la media de dias distintos es 5.

In [ ]:
modelo.loc[(modelo['label']==0)&(modelo['total_compras']>0)&(modelo['ultima_visita']>120)]['dias_distintos'].mean()

## MACHINE LEARNING

In [ ]:
##Ahora empieza ML

from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,roc_auc_score

## Split

In [ ]:
X = modelo.drop(['person','label'], axis=1)
y = modelo['label']

## K-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_2 = KMeans(n_clusters=2, init='k-means++', n_jobs=8).fit(X)
X['kmeans_2'] = kmeans_2.labels_

In [ ]:
kmeans_3 = KMeans(n_clusters=3, init='k-means++', n_jobs=8).fit(X)
X['kmeans_3'] = kmeans_3.labels_

In [ ]:
kmeans_4 = KMeans(n_clusters=4, init='k-means++', n_jobs=8).fit(X)
X['kmeans_4'] = kmeans_4.labels_

In [ ]:
kmeans_5 = KMeans(n_clusters=5, init='k-means++', n_jobs=8).fit(X)
X['kmeans_5'] = kmeans_5.labels_

In [ ]:
kmeans_50 = KMeans(n_clusters=50, init='k-means++', n_jobs=8).fit(X)
X['kmeans_50'] = kmeans_50.labels_

In [ ]:
kmeans_100 = KMeans(n_clusters=100, init='k-means++', n_jobs=8).fit(X)
X['kmeans_100'] = kmeans_100.labels_

## HDBScan

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=100)
cluster_labels = clusterer.fit_predict(X)
X['HDBScan'] = cluster_labels

## Feature importance con random forest

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
forest = ExtraTreesClassifier(n_estimators=500,
                              random_state=0,
                             class_weight ={1:8})
forest.fit(X, y)
def feature_importance(clasificador):
    importances = clasificador.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clasificador.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    plt.figure(figsize=(24,12))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()
feature_importance(forest)


## Select k best features

In [ ]:
from sklearn.feature_selection import SelectKBest
model= SelectKBest(k='all').fit(X,y)
columnas=list(X.columns[model.get_support()])
columnas

In [ ]:
model=model.transform(X)

In [ ]:
X = pd.DataFrame.from_records(model)
X.columns=columnas

In [ ]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2) 

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

### Graficador del ROC AUC 

In [ ]:
from sklearn.metrics import roc_curve, auc
def graficar_roc(clasificador):
    fpr = dict()
    tpr = dict()
    fpr, tpr, _ = roc_curve(y_test, clasificador.predict_proba(X_test)[:,1])
    roc_auc = dict()
    roc_auc = auc(fpr, tpr)

    # make the plot
    plt.figure(figsize=(10,10))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.plot(fpr, tpr, label='AUC = {0}'.format(roc_auc), linewidth =3)        
    plt.legend(loc="lower right", shadow=True, fancybox =True) 
    plt.show()

## Algoritmos de clasificación

## XGBOOST

In [ ]:
my_classifier1 = xgb.XGBClassifier(objective ='binary:logistic',max_depth =1,learning_rate = 0.02, n_estimators = 500,scale_pos_weight=7)
my_classifier1.fit(X_train,y_train)
predictions=my_classifier1.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

In [ ]:
from xgboost import plot_importance
ax = plot_importance(my_classifier1,height=0.5)
fig = ax.figure
fig.set_size_inches(20,10)
plt.show()

In [ ]:
ax = plot_importance(my_classifier1,height=0.5,importance_type='gain')
fig = ax.figure
fig.set_size_inches(20,10)
plt.show()

In [ ]:
ax = plot_importance(my_classifier1,height=0.5,importance_type='cover')
fig = ax.figure
fig.set_size_inches(20,10)
plt.show()

In [ ]:
graficar_roc(my_classifier1)

## LightGBM

In [ ]:
import lightgbm as lgb
my_classifier2 = lgb.LGBMClassifier(   
        objective='binary',
        boosting='goss',
        scale_pos_weight=8,
        n_estimators=13000,
        learning_rate=0.001,
        max_depth=1, 
        num_threads =6,        
    )

my_classifier2.fit(X_train,y_train)
predictions=my_classifier2.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier2.predict_proba(X_test)[:,1])

In [ ]:
graficar_roc(my_classifier2)

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier #a mayor depth, mas precisos los 0, menos los 1,class weight cambia mucho
my_classifier3 = RandomForestClassifier(criterion= 'entropy',max_depth= 5,n_estimators=1000, max_features= 'auto', class_weight={1:9} ,n_jobs=-1)
my_classifier3.fit(X_train,y_train)
predictions=my_classifier3.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier3.predict_proba(X_test)[:,1])

In [ ]:
graficar_roc(my_classifier3)

In [ ]:
feature_importance(my_classifier3)

## Cat Boost

In [ ]:
from catboost import CatBoostClassifier
my_classifier4=CatBoostClassifier(class_weights=[1, 8] ,logging_level='Silent',thread_count=6,boosting_type='Ordered',loss_function='Logloss',depth=3,leaf_estimation_method ='Newton',
                                 l2_leaf_reg = 23,od_pval=0.0000001)
my_classifier4.fit(X_train,y_train)
predictions=my_classifier4.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier4.predict_proba(X_test)[:,1])

y_pred=0.0
for i in range(10):
    clf=CatBoostClassifier(class_weights=[1, 8] ,logging_level='Silent',thread_count=6,boosting_type='Ordered',loss_function='Logloss',depth=3,leaf_estimation_method ='Newton',
                                 l2_leaf_reg = 23,od_pval=0.0000001,random_seed=i+200)
    fit_model = clf.fit(X_train,y_train)
    y_pred +=  fit_model.predict_proba(X_test)[:,1]
y_pred /= 10

In [ ]:
graficar_roc(my_classifier4)

## Voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[  ('XGB', my_classifier1), ('GBM', my_classifier2),('RF', my_classifier3),('CB', my_classifier4)], voting='soft', n_jobs =-1)
vclf = vclf.fit(X_train, y_train)
predictions = vclf.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

In [ ]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  vclf.predict_proba(X_test)[:,1])

In [ ]:
graficar_roc(vclf)

## KAGGLE PREDICTIONS

In [ ]:
kaggle= pd.read_csv('data/kaggle.csv',low_memory=False)
kaggle=pd.merge(kaggle, a_predecir, on='person', how='inner')

In [ ]:
kaggle2=kaggle.loc[:,columnas]
kaggle2 = kaggle2.fillna(0)
kaggle2.head()

In [ ]:
kmeans_2 = KMeans(n_clusters=2, init='k-means++', n_jobs=8).fit(kaggle2)
kaggle2['kmeans_2'] = kmeans_2.labels_
kmeans_3 = KMeans(n_clusters=3, init='k-means++', n_jobs=8).fit(kaggle2)
kaggle2['kmeans_3'] = kmeans_3.labels_
kmeans_4 = KMeans(n_clusters=4, init='k-means++', n_jobs=8).fit(kaggle2)
kaggle2['kmeans_4'] = kmeans_4.labels_
kmeans_5 = KMeans(n_clusters=5, init='k-means++', n_jobs=8).fit(kaggle2)
kaggle2['kmeans_5'] = kmeans_5.labels_

clusterer = hdbscan.HDBSCAN(min_cluster_size=100)
cluster_labels = clusterer.fit_predict(kaggle2)
kaggle2['HDBScan'] = cluster_labels

In [ ]:
vclf = VotingClassifier(estimators=[  ('XGB', my_classifier1), ('GBM', my_classifier2),('RF', my_classifier3),('CB', my_classifier4)], voting='soft', n_jobs =-1)
vclf = vclf.fit(X, y)

In [ ]:
predictions=vclf.predict_proba(kaggle2)

In [ ]:
kaggle['prediction0'] = predictions[:,0]
kaggle['label'] = predictions[:,1]

In [ ]:
kaggle.head(15)

In [ ]:
kaggle[['person','label']].to_csv('predicciones5.csv', encoding='utf-8', index=False)